In [2]:
import os
import sqlite3
import polars as pl

In [3]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(f'SQLITE version:', sqlite3.version)
    except sqlite3.Error as e:
        print(e)
    return conn

def create_table(conn, schema):
    try:
        c = conn.cursor()
        c.execute(schema)
    except sqlite3.Error as e:
        print(e)

In [4]:
def get_data(db_file, table_name):
    conn = create_connection(db_file)
    cursor = conn.cursor()

    df = pl.read_database(f"SELECT * FROM {table_name}", conn)#
    return conn, cursor, df

In [12]:
_, _, scan_df = get_data('./outputs/baselineScans.db', 'dicomdb')
scan_df.shape

SQLITE version: 2.6.0


(1862, 3)

In [5]:
_, _, spine_df = get_data('./outputs/baselineSpine.db', 'spinedb')
spine_df.shape

SQLITE version: 2.6.0


(1471, 5)

In [6]:
_, _, segment_df = get_data('./outputs/baselineSegmentation.db', 'segmentdb')
segment_df.shape

SQLITE version: 2.6.0


(1440, 5)

In [7]:
segment_df.select("status_code").unique()

status_code
i64
200
400
500


In [10]:
segment_df.filter(
    pl.col("status_code").is_in([500])
).shape


(27, 5)

### Find patients with a scout in the CT series

In [6]:
tmp = spine_df.filter(
    pl.col("status_code") == 200,
    ).join(segment_df, on=['experiment_id'], how='anti')

tmp.shape

(120, 5)

In [25]:
#tmp.write_csv("./outputs/boneScansInCTFolder.csv")

### Inspect errors

700 = no centroids; 800 = LoadImage error (doesn't think the dir. contains DICOMs);900=secondary image selected

In [7]:
errors = spine_df.filter(
    pl.col("status_code") != 200
)#.group_by("status_code")
errors.group_by("status_code").count()

status_code,count
i64,u32
900,6
800,1
700,24


In [35]:
errors.write_csv("./outputs/spineErrors.csv")